In [6]:
import org.apache.spark.SparkContext

import org.apache.spark.SparkContext


In [7]:
val sc = new SparkContext(master = "local[*]", appName = "MovieLens")

org.apache.spark.SparkException:  Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:

In [12]:
val rawdata = sc.textFile("u.data")

rawdata: org.apache.spark.rdd.RDD[String] = u.data MapPartitionsRDD[5] at textFile at <console>:26


In [13]:
// usernumber, recomended_movie_number, rating
rawdata.first()

res3: String = 196	242	3	881250949


In [16]:
val rawratings = rawdata.map(_.split("\t").take(3))

rawratings: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[7] at map at <console>:27


In [ ]:
// Creating an ML model starts from hear

In [17]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating

import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating


In [18]:
val ratings = rawratings.map{ 
    case Array(user, movie, rating) => Rating(user.toInt, movie.toInt, rating.toDouble)
}

ratings: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[8] at map at <console>:29


In [19]:
ratings.first()

res6: org.apache.spark.mllib.recommendation.Rating = Rating(196,242,3.0)


In [20]:
// Training the data
// parms:- rank=50, iteration=10, lambda=0.01

In [22]:
// Train the model
val model = ALS.train(ratings, 50, 10, 0.01)

model: org.apache.spark.mllib.recommendation.MatrixFactorizationModel = org.apache.spark.mllib.recommendation.MatrixFactorizationModel@231f9357


In [23]:
// Test the model
val predictedRating = model.predict(789, 123)

predictedRating: Double = 2.3526242759635414


In [25]:
// Generate recommendations
val userId = 789
val k = 50
val topKRecs = model.recommendProducts(userId,k)
println(topKRecs.mkString("\n"))

Rating(789,76,5.434948836491207)
Rating(789,39,5.378416728548247)
Rating(789,447,5.317798470221051)
Rating(789,502,5.14839473515516)
Rating(789,185,5.1373519907926966)
Rating(789,192,5.116849562116678)
Rating(789,518,5.114898181453012)
Rating(789,642,5.111137676354665)
Rating(789,693,5.0819041932052995)
Rating(789,127,5.012689437185902)
Rating(789,475,4.98846703822459)
Rating(789,276,4.988235698184952)
Rating(789,741,4.9688601070427705)
Rating(789,100,4.93640432974007)
Rating(789,179,4.927150051029097)
Rating(789,64,4.926808779889318)
Rating(789,9,4.913925122488399)
Rating(789,129,4.911489008309532)
Rating(789,150,4.9078524892213)
Rating(789,346,4.905457901596444)
Rating(789,50,4.887630547990506)
Rating(789,197,4.844415548068809)
Rating(789,47,4.841324252988163)
Rating(789,199,4.8043727121868915)
Rating(789,56,4.7783510160726905)
Rating(789,589,4.764352166391196)
Rating(789,87,4.761068430383609)
Rating(789,614,4.748048081344811)
Rating(789,182,4.747400468684088)
Rating(789,193,4.695709

userId: Int = 789
k: Int = 50
topKRecs: Array[org.apache.spark.mllib.recommendation.Rating] = Array(Rating(789,76,5.434948836491207), Rating(789,39,5.378416728548247), Rating(789,447,5.317798470221051), Rating(789,502,5.14839473515516), Rating(789,185,5.1373519907926966), Rating(789,192,5.116849562116678), Rating(789,518,5.114898181453012), Rating(789,642,5.111137676354665), Rating(789,693,5.0819041932052995), Rating(789,127,5.012689437185902), Rating(789,475,4.98846703822459), Rating(789,276,4.988235698184952), Rating(789,741,4.9688601070427705), Rating(789,100,4.93640432974007), Rating(789,179,4.927150051029097), Rating(789,64,4.926808779889318), Rating(789,9,4.913925122488399), Rating(789,129,4.911489008309532), Rating(789,150,4.9078524892213), Rating(789,346,4.905457901596444), Ra...


In [26]:
val movies = sc.textFile("u.item")

movies: org.apache.spark.rdd.RDD[String] = u.item MapPartitionsRDD[428] at textFile at <console>:28


In [27]:
val titles = movies.map(line => line.split("\\|").take(2)).map(x => (x(0).toInt, x(1))).collectAsMap()

titles: scala.collection.Map[Int,String] = Map(137 -> Big Night (1996), 891 -> Bent (1997), 550 -> Die Hard: With a Vengeance (1995), 1205 -> Secret Agent, The (1996), 146 -> Unhook the Stars (1996), 864 -> My Fellow Americans (1996), 559 -> Interview with the Vampire (1994), 218 -> Cape Fear (1991), 568 -> Speed (1994), 227 -> Star Trek VI: The Undiscovered Country (1991), 765 -> Boomerang (1992), 1115 -> Twelfth Night (1996), 774 -> Prophecy, The (1995), 433 -> Heathers (1989), 92 -> True Romance (1993), 1528 -> Nowhere (1997), 846 -> To Gillian on Her 37th Birthday (1996), 1187 -> Switchblade Sisters (1975), 1501 -> Prisoner of the Mountains (Kavkazsky Plennik) (1996), 442 -> Amityville Curse, The (1990), 1160 -> Love! Valour! Compassion! (1997), 101 -> Heavy Metal (1981), 1196 -> Sa...


In [31]:
titles(137 )

res12: String = Big Night (1996)


In [32]:
val moviesForUser = ratings.keyBy(_.user).lookup(789)

moviesForUser: Seq[org.apache.spark.mllib.recommendation.Rating] = WrappedArray(Rating(789,1012,4.0), Rating(789,127,5.0), Rating(789,475,5.0), Rating(789,93,4.0), Rating(789,1161,3.0), Rating(789,286,1.0), Rating(789,293,4.0), Rating(789,9,5.0), Rating(789,50,5.0), Rating(789,294,3.0), Rating(789,181,4.0), Rating(789,1,3.0), Rating(789,1008,4.0), Rating(789,508,4.0), Rating(789,284,3.0), Rating(789,1017,3.0), Rating(789,137,2.0), Rating(789,111,3.0), Rating(789,742,3.0), Rating(789,248,3.0), Rating(789,249,3.0), Rating(789,1007,4.0), Rating(789,591,3.0), Rating(789,150,5.0), Rating(789,276,5.0), Rating(789,151,2.0), Rating(789,129,5.0), Rating(789,100,5.0), Rating(789,741,5.0), Rating(789,288,3.0), Rating(789,762,3.0), Rating(789,628,3.0), Rating(789,124,4.0))


In [33]:
println(moviesForUser.size)

33


In [34]:
val temp = moviesForUser.sortBy(_.rating).take(10).map(rating => (titles(rating.product), rating.rating))

temp.foreach(println)

(English Patient, The (1996),1.0)
(Big Night (1996),2.0)
(Willy Wonka and the Chocolate Factory (1971),2.0)
(Palookaville (1996),3.0)
(Liar Liar (1997),3.0)
(Toy Story (1995),3.0)
(Tin Cup (1996),3.0)
(Trees Lounge (1996),3.0)
(Truth About Cats & Dogs, The (1996),3.0)
(Ransom (1996),3.0)


temp: Seq[(String, Double)] = ArrayBuffer((English Patient, The (1996),1.0), (Big Night (1996),2.0), (Willy Wonka and the Chocolate Factory (1971),2.0), (Palookaville (1996),3.0), (Liar Liar (1997),3.0), (Toy Story (1995),3.0), (Tin Cup (1996),3.0), (Trees Lounge (1996),3.0), (Truth About Cats & Dogs, The (1996),3.0), (Ransom (1996),3.0))


### medium Movie Lense recomendation

In [35]:
//Import other necessary packages
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating
import org.apache.spark.SparkConf

import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating
import org.apache.spark.SparkConf


In [50]:
object Movie {
    def main(args: Array[String]) {

    val conf = new SparkConf().setAppName("Movie").setMaster("local[2]")
    val sc = new SparkContext(conf)
    val rawData = sc.textFile(" *Read Data from Movie CSV file* ")

    //rawData.first()
    val rawRatings = rawData.map( *Split rawData on tab delimiter* )
    val ratings = rawRatings.map { /*Map case array of User, Movie and Rating*/ }

    //Training the data
    val model = ALS.train(ratings, 50, 5, 0.01)
    model.userFeatures
    model.userFeatures.count
    model.productFeatures.count
    val predictedRating = {Predict for User 789 for movie 123}
    val userId = *User 789*
    val K = 10
    val topKRecs = model.recommendProducts( *Recommend for User for the particular value of K* )
    println(topKRecs.mkString("\n"))
    val movies = sc.textFile(" *Read Movie List Data* ")
    val titles = movies.map(line => line.split("\\|").take(2)).map(array => (array(0).toInt,array(1))).collectAsMap()
    val titlesRDD = movies.map(line => line.split("\\|").take(2)).map(array => (array(0).toInt,array(1))).cache()
    titles(123)
    val moviesForUser = ratings.*Search for User 789*
    val sqlContext= *Create SQL Context*
    val moviesRecommended = sqlContext.*Make a DataFrame of recommended movies*
    moviesRecommended.registerTempTable("moviesRecommendedTable")
    sqlContext.sql("Select count(*) from moviesRecommendedTable").foreach(println)
    moviesForUser. *Sort the ratings for User 789* .map( *Map the rating to movie title* ). *Print the rating*
    val results = moviesForUser.sortBy(-_.rating).take(30).map(rating => (titles(rating.product), rating.rating))
 }

<console>: 18: error: ';' expected but 'for' found.

....